The previous notebook (inception_resnet_v2_self_trained_on_200x150) created "baseline models" that I can improve on. For example I haven’t used the tabular data and neither have I used any image augmentation. Also, the original architecture likes square images, but I’m giving it rectangular ones. ROI detection net will have to be trained to enable the possibility to detect the main location of the lesion and crop the image accordingly. Both of these points will be addressed next, and this notebook is about improving the results by using tabular data.

In [1]:
import os
import absl.logging

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
base_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_splitted')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

Creating the tabular data:

In [15]:
def normalize(data: np.ndarray) -> np.ndarray:
    return (data - np.min(data)) / (np.max(data) - np.min(data))


metadata_path = os.path.join('..', 'data', 'HAM10000_metadata.csv')
data = pd.read_csv(metadata_path)
data['age'] = data.groupby('sex')['age'].transform(lambda x: x.fillna(x.mean()))
train_names = set(get_names(train_dir))
valid_names = set(get_names(valid_dir))
relevant_cols = ['dx', 'age', 'sex', 'localization']
train_df = data[data['image_id'].isin(train_names)].sort_values(by='image_id')[relevant_cols]
valid_df = data[data['image_id'].isin(valid_names)].sort_values(by='image_id')[relevant_cols]
train_age = np.expand_dims(normalize(train_df['age'].to_numpy()), -1)
train_sex_categories = pd.get_dummies(train_df['sex']).to_numpy()
train_localization_categories = pd.get_dummies(train_df['localization']).to_numpy()
valid_age = np.expand_dims(normalize(valid_df['age'].to_numpy()), -1)
valid_sex_categories = pd.get_dummies(valid_df['sex']).to_numpy()
valid_localization_categories = pd.get_dummies(valid_df['localization']).to_numpy()
X_train = np.hstack((train_age, train_sex_categories, train_localization_categories))
y_train = pd.get_dummies(train_df['dx'])
X_valid = np.hstack((valid_age, valid_sex_categories, valid_localization_categories))
y_valid = pd.get_dummies(valid_df['dx'])

Creating the image data:

In [16]:
def preprocess_dataset(dataset: tf.data.Dataset) -> tf.data.Dataset:
    rescale = keras.layers.Rescaling(1./255)

    return dataset.map(lambda image: rescale(image))


train_generator = keras.utils.image_dataset_from_directory(
    train_dir,
    labels=None,
    label_mode=None,
    image_size=(150, 200),
    batch_size=64,
    shuffle=False) # setting this param to None because the labels will be taken care of separately
valid_generator = keras.utils.image_dataset_from_directory(
    valid_dir,
    labels=None,
    label_mode=None,
    image_size=(150, 200),
    batch_size=64,
    shuffle=False) # setting this param to None because the labels will be taken care of separately
train_generator = preprocess_dataset(train_generator)
valid_generator = preprocess_dataset(valid_generator)

Found 8015 files belonging to 1 classes.
Found 2000 files belonging to 1 classes.


In [17]:
train_metadata_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
valid_metadata_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(64)
train_dataset = tf.data.Dataset.zip((train_generator, train_metadata_ds))
valid_dataset = tf.data.Dataset.zip((valid_generator, valid_metadata_ds))


def restructure(image_data, tabular_data_and_labels):
    tabular_data, labels = tabular_data_and_labels

    return (image_data, tabular_data), labels


train_dataset = train_dataset.map(restructure)
valid_dataset = valid_dataset.map(restructure)

In [20]:
SMALLER_WIDTH = 600 // 3
SMALLER_HEIGHT = 450 // 3
METADATA_COLS_COUNT = X_train.shape[1]


def get_model() -> keras.Model:
    base_model = InceptionResNetV2(include_top=False, weights=None, input_shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))
    img_x = base_model.output
    img_x = keras.layers.Dropout(.4)(img_x)
    img_x = keras.layers.GlobalAveragePooling2D()(img_x)
    img_x = keras.layers.Dense(512)(img_x)
    img_x = keras.layers.PReLU()(img_x)
    img_x = keras.layers.Dropout(.4)(img_x)
    img_x = keras.layers.Dense(512)(img_x)
    img_x = keras.layers.PReLU()(img_x)
    tab_input = keras.Input(shape=(METADATA_COLS_COUNT,), name='tab_input')
    tab_x = keras.layers.Dense(128)(tab_input)
    tab_x = keras.layers.PReLU()(tab_x)
    tab_x = keras.layers.Dense(128)(tab_x)
    tab_x = keras.layers.PReLU()(tab_x)
    img_x = keras.layers.Dropout(.4)(img_x)
    combined = keras.layers.concatenate([img_x, tab_x])
    predictions = keras.layers.Dense(7, activation='softmax')(combined)
    model = keras.Model(inputs=[base_model.input, tab_input], outputs=predictions)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [21]:
def run_model(model_factory, model_name: str) -> None:
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                                   min_delta=1e-6)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath='models/' + model_name + '_{epoch}',
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=f'tensor_logs/{model_name}')
    model = model_factory()

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=50,
        callbacks=[early_stopping, model_checkpoint, tensor_board])

In [22]:
run_model(get_model, f'images_original_inception_resnet_v2_200x150_with_tabular3')

Epoch 1/50
126/126 [==============================] - ETA: 0s - loss: 0.9338 - accuracy: 0.6841INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_1\assets


INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_1\assets


126/126 [==============================] - 136s 953ms/step - loss: 0.9338 - accuracy: 0.6841 - val_loss: 1.1889 - val_accuracy: 0.6520
Epoch 2/50
126/126 [==============================] - 54s 426ms/step - loss: 0.7473 - accuracy: 0.7275 - val_loss: 1.3337 - val_accuracy: 0.5440
Epoch 3/50
126/126 [==============================] - 54s 427ms/step - loss: 0.7196 - accuracy: 0.7351 - val_loss: 1.3653 - val_accuracy: 0.4760
Epoch 4/50
126/126 [==============================] - ETA: 0s - loss: 0.7035 - accuracy: 0.7384INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_4\assets


INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_4\assets


126/126 [==============================] - 115s 915ms/step - loss: 0.7035 - accuracy: 0.7384 - val_loss: 1.0003 - val_accuracy: 0.7185
Epoch 5/50
126/126 [==============================] - ETA: 0s - loss: 0.6723 - accuracy: 0.7513INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_5\assets


...

126/126 [==============================] - 115s 913ms/step - loss: 0.6723 - accuracy: 0.7513 - val_loss: 0.9436 - val_accuracy: 0.6445
Epoch 6/50
126/126 [==============================] - ETA: 0s - loss: 0.6524 - accuracy: 0.7581INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_6\assets


INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_6\assets


126/126 [==============================] - 113s 899ms/step - loss: 0.6524 - accuracy: 0.7581 - val_loss: 0.7924 - val_accuracy: 0.7225
Epoch 7/50
126/126 [==============================] - ETA: 0s - loss: 0.6314 - accuracy: 0.7653INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_7\assets


INFO:tensorflow:Assets written to: models\images_original_inception_resnet_v2_200x150_with_tabular3_7\assets


126/126 [==============================] - 115s 916ms/step - loss: 0.6314 - accuracy: 0.7653 - val_loss: 0.7645 - val_accuracy: 0.7065
Epoch 8/50
126/126 [==============================] - 54s 429ms/step - loss: 0.6195 - accuracy: 0.7668 - val_loss: 0.7793 - val_accuracy: 0.6995
Epoch 9/50
126/126 [==============================] - 54s 426ms/step - loss: 0.6053 - accuracy: 0.7747 - val_loss: 0.7864 - val_accuracy: 0.7240
Epoch 10/50
126/126 [==============================] - 54s 425ms/step - loss: 0.5874 - accuracy: 0.7842 - val_loss: 0.8389 - val_accuracy: 0.7040
Epoch 11/50
126/126 [==============================] - 54s 426ms/step - loss: 0.5783 - accuracy: 0.7840 - val_loss: 0.9180 - val_accuracy: 0.6930
Epoch 12/50
126/126 [==============================] - 54s 426ms/step - loss: 0.5692 - accuracy: 0.7860 - val_loss: 0.9417 - val_accuracy: 0.7045
Epoch 13/50
126/126 [==============================] - 54s 427ms/step - loss: 0.5707 - accuracy: 0.7904 - val_loss: 0.9243 - val_accuracy

And the final verdict is that using tabular data makes the network do worse than it would without it. It may have a potentially logical explanation - the tabular data is really simple and consists of age, sex and lesion location. Each of those even combined with each other is a weak indicator for skin cancer and some additional tabular data would have to be used to actually improve the CNN result.